# My COVID-19 Dashboard

My COVID-19 Dashboard focuses on generating an interactive bar chart displaying the number of female and male cases associated with a distribution of age ranges. Extending this initial plot, I identified 3 key dates associated with various milestones of UK's COVID situation, and added interactivity in the bar chart to display the respective data associated with these key dates. The 3 dates I chose to be of importance are as follows:

1) 23 March 2020 - Johnson announced the "biggest lockdown of society in British history" on national television. Social distancing was made legally mandatory and furlough scheme announced.

2) 4 January 2021 - Johnson announced new national lockdown measures for England due to the spread of another variant, later known as Alpha, that he described as “both frustrating and alarming

3) 19 July 2021 - England celebrated “Freedom Day” as the vast majority of Covid-19 restrictions were finally lifted



In [3]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

# 

In [4]:
# Loading of canned data
with open("england23March2020Data.json", "rt") as INFILE:
    initialEngland23March2020AgeGenderData=json.load(INFILE)
with open("england04Jan2021Data.json", "rt") as INFILE:
    initialEngland04Jan2021AgeGenderData=json.load(INFILE)
with open("england19Jul2021Data.json", "rt") as INFILE:
    initialEngland19Jul2021AgeGenderData=json.load(INFILE)


In [5]:
def wrangle_data(AgeDistributionData):
    """ Parameters: - data from json file or API call. Returns a dataframe. """
    dataDictionary=AgeDistributionData[0] # data['data'] is a list
    maleCases=dataDictionary['males']
    femaleCases=dataDictionary['females']
    ageranges=[x['age'] for x in maleCases]

    def min_age(agerange):
        agerange=agerange.replace('+','') 
        start=agerange.split('_')[0]
        return int(start)

    ageranges.sort(key=min_age)

    DataFrame=pd.DataFrame(index=ageranges, columns=['males','females', 'total'])

    for entry in maleCases: # each entry is a dictionary
        ageband=entry['age'] # our index position
        DataFrame.loc[ageband, 'males']=entry['value']
        for entry in femaleCases:
            ageband=entry['age']
            DataFrame.loc[ageband, 'females']=entry['value']
        DataFrame['total']=DataFrame['males']+DataFrame['females']
    return DataFrame

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
england23March2020AgeGenderDataFrame=wrangle_data(initialEngland23March2020AgeGenderData)
england04Jan2021AgeGenderDataFrame=wrangle_data(initialEngland04Jan2021AgeGenderData) 
england19Jul2021AgeGenderDataFrame=wrangle_data(initialEngland19Jul2021AgeGenderData)  


In [6]:
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """   
    englandFilter = [
        'areaType=nation',
        'areaName=England'
    ]

    structure = {
        "date":"date",
        "males": "maleCases",
        "females": "femaleCases"
    }

    englandAPI = Cov19API(filters=englandFilter, structure=structure)

    updatedEnglandData=englandAPI.get_json()

    updatedEngland23March2020Data = []
    updatedEngland04Jan2021Data = []
    updatedEngland19Jul2021Data = []

    for i in range(1000):
        if updatedEnglandData['data'][i]['date'] == "2020-03-23":
            updatedEngland23March2020Data.append(updatedEnglandData['data'][i])
        elif updatedEnglandData['data'][i]['date'] ==  "2021-01-04":
            updatedEngland04Jan2021Data.append(updatedEnglandData['data'][i])
        elif updatedEnglandData['data'][i]['date'] == "2021-07-19":
            updatedEngland19Jul2021Data.append(updatedEnglandData['data'][i])

 
    return [updatedEngland23March2020Data, updatedEngland04Jan2021Data, updatedEngland19Jul2021Data]

  



  

In [7]:
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    try:
        englandDataInDateOrder = access_api()
    except Exception:
        print("Sorry, there was difficulty connecting to UK.GOVs Open Data API. Please try again later.")
    
    global england23March2020AgeGenderDataFrame
    global england04Jan2021AgeGenderDataFrame
    global england19Jul2021AgeGenderDataFrame

    england23March2020AgeGenderDataFrame=wrangle_data(englandDataInDateOrder[0])
    england04Jan2021AgeGenderDataFrame=wrangle_data(englandDataInDateOrder[1])
    england19Jul2021AgeGenderDataFrame=wrangle_data(englandDataInDateOrder[2])

    apibutton.icon="check"
    apibutton.disabled=False


apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

 
apibutton.on_click(api_button_callback) # the name of your function inside these brackets
display(apibutton)



Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

## Graphs and Analysis

In [8]:
agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'], # options available
    value=['males', 'females'], # initial value
    rows=3, # rows of the selection box
    description='Gender',
    disabled=False
)

date=wdg.RadioButtons(
    options=['23rd March 2020',
            '4th January 2021', 
            '19th July 2021'],
    value='23rd March 2020',
    description='Date:',
    disabled=False
)

def age_graph(graphcolumns, datecolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if datecolumns == '23rd March 2020':
        if ncols>0:
            england23March2020AgeGenderDataFrame.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
            plt.show() # important - graphs won't update properly if this is missing
        else:
            # if the user has not selected any column, print a message instead
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")
    if datecolumns == '4th January 2021':
        if ncols>0:
            england04Jan2021AgeGenderDataFrame.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
            plt.show() # important - graphs won't update properly if this is missing
        else:
            # if the user has not selected any column, print a message instead
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")
    if datecolumns == '19th July 2021':
        if ncols>0:
            england19Jul2021AgeGenderDataFrame.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
            plt.show() # important - graphs won't update properly if this is missing
        else:
            # if the user has not selected any column, print a message instead
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols, 'datecolumns': date})


ctrls = wdg.VBox([agecols, date])
form=wdg.HBox([output, ctrls])

display(form)





**Author and Copyright Notice** Covid-19 Dashboard (C) Nicolas Robinson, 2022 (ec221130@qmul.ac.uk - web). All rights reserved.